# MIMIC-III M3Care Baseline

## Initialize Dataset

In [1]:
from pathlib import Path

from mimic.datasets.mimic import MimicDataset

PROCESSED_DIR = Path('./mimic/data/processed')

dataset_paths = {
    'demographic': PROCESSED_DIR / 'test/demographic.csv',
    'vitals': PROCESSED_DIR / 'test/vitals.csv',
    'interventions': PROCESSED_DIR / 'test/interventions.csv',
    'static_notes': PROCESSED_DIR / 'test/notes_static.h5',
    'ts_notes': PROCESSED_DIR / 'test/notes_ts.h5'
}

auxillary_paths = {
    'pat_ids': PROCESSED_DIR / 'test_idxs.npy',
    'vocab': PROCESSED_DIR / 'vocab.json'
}

ds = MimicDataset(dataset_paths, auxillary_paths)

## Initialize Models

In [2]:
from m3care.m3care import Modal, M3Care

EMB_DIM = 512
OUT_DIM = 2
DROPOUT = 0.3

DEM_INPUT = 18

VIT_INPUT = 104
VIT_TIME_DIM = 150

ITV_INPUT = 14
ITV_TIME_DIM = 150

NOTES_TOK_DIM = 16
NOTES_VOCAB_SIZE = 5000
NOTES_NUM_HEADS = 8

NST_WORD_LIM = 10000

NTS_TIME_DIM = 128
NTS_WORD_LIM = 5000

In [3]:
from mimic.models.vector import MLP
from mimic.models.sequential import SequentialEmbedder
from mimic.models.nlp import NLSequenceEmbedder, NLTimeSeriesSequenceEmbedder

dem_model = MLP(in_dim=DEM_INPUT,
                hidden_dim=[128,192,256],
                out_dim=EMB_DIM, 
                bias=True,
                relu=True,
                norm=True)

vit_model = SequentialEmbedder(feat_dim=VIT_INPUT,
                             embed_dim=EMB_DIM,
                             max_len=VIT_TIME_DIM,
                             dropout=DROPOUT)

itv_model = SequentialEmbedder(feat_dim=ITV_INPUT,
                             embed_dim=EMB_DIM,
                             max_len=ITV_TIME_DIM,
                             dropout=DROPOUT)

nst_model = NLSequenceEmbedder(vocab=ds.vocab,
                               token_dim=NOTES_TOK_DIM,
                               embed_dim=EMB_DIM,
                               num_heads=NOTES_NUM_HEADS,
                               vocab_size=NOTES_VOCAB_SIZE,
                               max_len=NST_WORD_LIM,
                               dropout=DROPOUT)

nts_model = NLTimeSeriesSequenceEmbedder(vocab=ds.vocab,
                                         token_dim=NOTES_TOK_DIM,
                                         embed_dim=EMB_DIM,
                                         num_heads=NOTES_NUM_HEADS,
                                         vocab_size=NOTES_VOCAB_SIZE,
                                         max_len=NTS_WORD_LIM,
                                         dropout=DROPOUT)

In [4]:
modals = [
    Modal('demographic', dem_model, False, None),   
    Modal('vitals', vit_model, True, VIT_TIME_DIM),
    Modal('interventions', itv_model, True, ITV_TIME_DIM),
    Modal('static_notes', nst_model, True, None),
    Modal('ts_notes', nts_model, True, NTS_TIME_DIM)
]

model = M3Care(modals, EMB_DIM, OUT_DIM, 'cpu')

c:\Users\Jason\Work\M3Care\env\lib\site-packages\torch\nn\modules\transformer.py:307: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.num_heads is odd
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


In [5]:
x, masks = ds[0:10]

In [6]:
y = model(x, masks, 10)

--------------------
TENLIST: embs_orig
demographic torch.Size([10, 512]) torch.float32
vitals torch.Size([10, 150, 512]) torch.float32
interventions torch.Size([10, 150, 512]) torch.float32
static_notes torch.Size([10, 512]) torch.float32
ts_notes torch.Size([10, 128, 512]) torch.float32
--------------------
--------------------
TENLIST: emb_masks_orig
demographic torch.Size([10]) torch.bool
vitals torch.Size([10, 150]) torch.bool
interventions torch.Size([10, 150]) torch.bool
static_notes torch.Size([10]) torch.bool
ts_notes torch.Size([10, 128]) torch.bool
--------------------
--------------------
TENLIST: embs
demographic torch.Size([10, 512]) torch.float32
vitals torch.Size([10, 512]) torch.float32
interventions torch.Size([10, 512]) torch.float32
static_notes torch.Size([10, 512]) torch.float32
ts_notes torch.Size([10, 512]) torch.float32
--------------------
--------------------
TENLIST: emb_masks
demographic torch.Size([10]) torch.bool
vitals torch.Size([10]) torch.bool
interve

In [7]:
y

(tensor([[-0.1189,  0.0823],
         [ 0.0897,  0.2886],
         [-0.0008,  0.2745],
         [-0.0784,  0.3836],
         [-0.0178,  0.4017],
         [-0.0545,  0.0263],
         [-0.0426,  0.0478],
         [ 0.1427,  0.3587],
         [ 0.0054,  0.2173],
         [-0.1194,  0.3866]], grad_fn=<SqueezeBackward1>),
 tensor(20145.2637, grad_fn=<AddBackward0>))